In [1]:
#Task 2.1-User-based/Item-based Collaborative Filtering
#By Liping Li and Saixiong Han

#import train_user_view data
import pandas as pd
import numpy as np

train_user_view_df = pd.read_table('C:/Study/assignment8/train_user_view.txt')
train_user_view_df.head()

,user_id,talk_id,view_time
0,1,10474,07/27/2016 12:27:35
1,1,10488,08/05/2016 12:34:01
2,1,10506,08/07/2016 15:55:57
3,1,10446,08/07/2016 15:58:09
4,1,10506,08/07/2016 16:06:26


In [2]:
new_df = train_user_view_df.sort_values(['user_id','talk_id','view_time'],ascending=[True,True,True])
new_df.head()

,user_id,talk_id,view_time
3,1,10446,08/07/2016 15:58:09
5,1,10446,08/08/2016 18:18:46
6,1,10446,08/09/2016 10:38:38
7,1,10446,08/09/2016 10:39:33
8,1,10446,08/09/2016 16:33:56


In [3]:
#convert the train_user_view.txt into a user-talk matrix.
n_users = train_user_view_df.user_id.unique().shape[0]
n_talks = train_user_view_df.talk_id.unique().shape[0]
print str(n_users) + ' users'
print str(n_talks) + ' talks'
ctr =0
ctr2 = 0
temp = 0
comments = np.zeros((n_users, n_talks))
for row in new_df.itertuples():
    if (row[1]-1)!=temp:
        ctr = 0
        ctr2 = ctr2+1
    comments[ctr2, ctr] = row[2]
    if (ctr+1)<n_talks:
        ctr += 1
    else: 
        ctr = 0
    temp = row[1]-1
comments

37 users
165 talks


array([[ 10446.,  10446.,  10446., ...,      0.,      0.,      0.],
       [ 10636.,  10636.,  10636., ...,  10634.,  10636.,  10636.],
       [ 10516.,  10517.,  10532., ...,      0.,      0.,      0.],
       ..., 
       [ 10634.,      0.,      0., ...,      0.,      0.,      0.],
       [ 10666.,  10666.,  10666., ...,      0.,      0.,      0.],
       [ 10583.,  10591.,  10702., ...,      0.,      0.,      0.]])

In [4]:
sparsity = float(len(comments.nonzero()[0]))
sparsity /= (comments.shape[0] * comments.shape[1])
sparsity *= 100
print 'Sparsity: {:4.2f}%'.format(sparsity)

Sparsity: 20.44%


In [5]:
#split the data into 80/10/10 trainning/validation/test data

def train_rest_split(comments):
    rest = np.zeros(comments.shape)
    train = comments.copy()
    for user in xrange(comments.shape[0]):
        rest_comments = np.random.choice(comments[user, :].nonzero()[0], 
                                        size=20, 
                                        replace=True)
        train[user, rest_comments] = 0.
        rest[user, rest_comments] = comments[user, rest_comments]
        
    # Test and training are truly disjoint
    assert(np.all((train * rest) == 0)) 
    return train, rest

train, rest = train_rest_split(comments)

def valida_test_split(rest):
    test = np.zeros(rest.shape)
    valida = rest.copy()
    for user in xrange(rest.shape[0]):
        test_comments = np.random.choice(rest[user, :].nonzero()[0], 
                                        size=50, 
                                        replace=True)
        valida[user, test_comments] = 0.
        test[user, test_comments] = rest[user, test_comments]
        
    # Test and training are truly disjoint
    assert(np.all((train * rest) == 0)) 
    return valida, test

valida, test = valida_test_split(rest)

In [6]:
#Item-based Collaborative Filtering
#item-similarity matrix will measure the similarity between any two pairs of items.
def slow_similarity(comments, kind='user'):
    if kind == 'user':
        axmax = 0
        axmin = 1
    elif kind == 'talk':
        axmax = 1
        axmin = 0
    sim = np.zeros((comments.shape[axmax], comments.shape[axmax]))
    for u in xrange(comments.shape[axmax]):
        for uprime in xrange(comments.shape[axmax]):
            rui_sqrd = 0.
            ruprimei_sqrd = 0.
            for i in xrange(comments.shape[axmin]):
                sim[u, uprime] = comments[u, i] * comments[uprime, i]
                rui_sqrd += comments[u, i] ** 2
                ruprimei_sqrd += comments[uprime, i] ** 2
            sim[u, uprime] /= rui_sqrd * ruprimei_sqrd
    return sim

def fast_similarity(comments, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = comments.dot(comments.T) + epsilon
    elif kind == 'talk':
        sim = comments.T.dot(comments) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [7]:
%timeit fast_similarity(train, kind='user')

The slowest run took 722.61 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 38.4 µs per loop


In [8]:
%timeit slow_similarity(train, kind='user')

c:\python27\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in double_scalars


1 loop, best of 3: 286 ms per loop


In [9]:
user_similarity_f = fast_similarity(train, kind='user')
talk_similarity_f = fast_similarity(train, kind='talk')
print talk_similarity_f[:4, :4]

[[ 1.          0.7078604   0.69358752  0.92858451]
 [ 0.7078604   1.          0.54638255  0.70809208]
 [ 0.69358752  0.54638255  1.          0.69376149]
 [ 0.92858451  0.70809208  0.69376149  1.        ]]


In [10]:
#user_based collaborating filtering

def predict_slow_simple(comments, similarity, kind='user'):
    pred = np.zeros(comments.shape)
    if kind == 'user':
        for i in xrange(comments.shape[0]):
            for j in xrange(comments.shape[1]):
                pred[i, j] = similarity[i, :].dot(comments[:, j])\
                             /np.sum(np.abs(similarity[i, :]))
        return pred
    elif kind == 'talk':
        for i in xrange(comments.shape[0]):
            for j in xrange(comments.shape[1]):
                pred[i, j] = similarity[j, :].dot(comments[i, :].T)\
                             /np.sum(np.abs(similarity[j, :]))

        return pred

def predict_fast_simple(comments, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(comments) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'talk':
        return comments.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [11]:
%timeit predict_slow_simple(train, user_similarity_f, kind='user')

10 loops, best of 3: 34 ms per loop


In [12]:
%timeit predict_fast_simple(train, user_similarity_f, kind='user')

The slowest run took 6.48 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 42.4 µs per loop


In [13]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [14]:
#get result from training dataset and validation dataset

talk_prediction = predict_fast_simple(train, talk_similarity_f, kind='talk')
user_prediction = predict_fast_simple(train, user_similarity_f, kind='user')

print 'User-based CF MSE: ' + str(get_mse(user_prediction, valida))
print 'Talk-based CF MSE: ' + str(get_mse(talk_prediction, valida))

User-based CF MSE: 35957120.4698
Talk-based CF MSE: 37266139.3729


In [15]:
#get result from training dataset and test dataset

talk_prediction = predict_fast_simple(train, talk_similarity_f, kind='talk')
user_prediction = predict_fast_simple(train, user_similarity_f, kind='user')

print 'User-based CF MSE: ' + str(get_mse(user_prediction, test))
print 'Talk-based CF MSE: ' + str(get_mse(talk_prediction, test))

User-based CF MSE: 50002156.9582
Talk-based CF MSE: 71853515.2318


In [16]:
#Task 2.2-Matrix Factorization
#define the matrix_factorization
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T


In [15]:
#Get the matrix factorization of training data

N = len(train)
M = len(train[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(train, P, Q, K)
nR = numpy.dot(nP, nQ.T)

c:\python27\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: overflow encountered in double_scalars


In [16]:
print nR

[[        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,  0.61453516         nan
          nan]
 ..., 
 [        nan         nan         nan ...,  0.70864507         nan
          nan]
 [        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,  1.00379919         nan
          nan]]


In [17]:
#Since I don't know how to use the example to get the accuracy of training data and test data
#I used a sklearn package called Non-Negative Matrix Factorization (NMF)
#This way I can get the accuracy at least
X = np.array(train)
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
model.fit(X) 
NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [18]:
#Get the matrix factorization
model.components_

array([[ 38.33969294,  34.69165073,  42.87668592,  38.55081422,
         32.04162031,  42.77534471,  34.32147887,  39.38313969,
         33.93625288,  28.18142585,  29.46942033,  41.6706407 ,
         41.01902362,  43.62743459,  12.85758172,  33.24884706,
         35.77679586,  35.0697303 ,  34.43292329,  40.58168399,
         37.81703833,  37.90633121,  32.78898653,  38.16885223,
         26.4940717 ,  40.87498102,  38.17015081,  14.00206635,
         45.87471204,  33.93093601,  32.47709909,  31.93019302,
         44.28623686,  31.45915653,  42.42254232,  40.33598192,
         20.14837335,  31.42075325,  44.07116725,  30.32700458,
         29.45755887,  33.50569824,  44.67754267,  44.69511153,
         32.4775929 ,  31.88445041,  46.56806066,  46.53374968,
         48.72819182,  21.53290787,  35.91448879,  44.74938261,
         46.58876758,  46.91414814,  46.91892995,  48.78792413,
         48.80929927,  44.79691941,  46.66710898,  30.74193021,
         46.67381605,  33.82348669,  10.

In [19]:
#The accuracy of the training data and test data
model.reconstruction_err_

155369.56209961325

In [22]:
X = np.array(valida)
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
model.fit(X) 
NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [23]:
#The accuarcy of validation data
model.reconstruction_err_

31256.901825180175